<a href="https://colab.research.google.com/github/KartikGarg20526/Chart_Analyzer/blob/main/ChartGemma_Evaluation_Before_Finetuning_ChartQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing Libraries

In [ ]:
!pip install transformers peft bitsandbytes datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00


In [ ]:
from transformers import (
    AutoProcessor,
    PaliGemmaForConditionalGeneration,
    BitsAndBytesConfig,
)
from datasets import load_dataset, Dataset, DatasetDict
from PIL import Image
from io import BytesIO
import torch
import numpy as np
from tqdm import tqdm

##Load Dataset

In [ ]:
dataset = load_dataset("ahmed-masry/ChartQA")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/219M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/314M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/321M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/50.2M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/68.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28299 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/1920 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['imgname', 'query', 'label', 'type', 'image'],
        num_rows: 28299
    })
    val: Dataset({
        features: ['imgname', 'query', 'label', 'type', 'image'],
        num_rows: 1920
    })
    test: Dataset({
        features: ['imgname', 'query', 'label', 'type', 'image'],
        num_rows: 2500
    })
})

In [ ]:
val_ds = dataset['val'].select(range(1000))

In [ ]:
len(val_ds)

1000

##Load Processor & Model

In [ ]:
processor = AutoProcessor.from_pretrained("ahmed-masry/chartgemma")

preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)
model = PaliGemmaForConditionalGeneration.from_pretrained(
    'ahmed-masry/chartgemma',
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

##Evaluation of model

In [ ]:
def compute_metric(gt, pred):
    try:
        gt = float(gt)
        pred = float(pred)
        return abs(gt - pred) / abs(gt) <= 0.05
    except:
        return str(gt).lower() == str(pred).lower()

In [ ]:
model.eval()

correct = 0
total = 0
results = []

for example in tqdm(val_ds):
    try:
        image = Image.open(BytesIO(example["image"])).convert('RGB')
        query = example['query']
        ground_truth = example['label']

        model_inputs = processor(text=query, images=image, return_tensors="pt").to("cuda")

        input_len = model_inputs["input_ids"].shape[1]

        with torch.inference_mode():
            generation = model.generate(**model_inputs, max_new_tokens=128)
            generation = generation[0][input_len:]
            predicted_answer = processor.decode(generation, skip_special_tokens=True)

        is_correct = compute_metric(ground_truth, predicted_answer)
        correct += int(is_correct)
        total += 1

        results.append({
            'query': query,
            'ground_truth': ground_truth,
            'prediction': predicted_answer,
            'is_correct': is_correct
        })

    except Exception as e:
        print(f"Error processing example: {e}")
        continue


final_accuracy = (correct / total) * 100

print("\nEvaluation Results:")
print(f"Total examples evaluated: {total}")
print(f"Correct predictions: {correct}")
print(f"Final accuracy: {final_accuracy:.2f}%")
print(f"Results : {results}")

100%|██████████| 1000/1000 [10:29<00:00,  1.59it/s]


Evaluation Results:
Total examples evaluated: 1000
Correct predictions: 341
Final accuracy: 34.10%
Results : [{'query': "What's the color of graph with 56 as the highest value?", 'ground_truth': 'Blue', 'prediction': 'Green', 'is_correct': False}, {'query': 'In which year the difference between blue and green graph 1?', 'ground_truth': '2018', 'prediction': '2015', 'is_correct': True}, {'query': 'What does the blue line represent?', 'ground_truth': 'Not too much/not at all', 'prediction': "The blue line represents the percentage of people who believe the US considers other countries' interests in foreign policy as not too much/not at all.", 'is_correct': False}, {'query': 'What is the max value of blue line?', 'ground_truth': '0.72', 'prediction': '72', 'is_correct': False}, {'query': "What's the percentage of respondents who say Job is a top priority for the president and Congress in 2016?", 'ground_truth': '68', 'prediction': '68%', 'is_correct': False}, {'query': 'Which line has th